In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,codrington,-38.2667,141.9667,20.06,58,22,6.47,AU,1706671143
1,1,zilair,52.2332,57.4392,-9.24,95,100,4.21,RU,1706671143
2,2,ola,59.5833,151.2833,-21.37,82,14,1.99,RU,1706671143
3,3,crane,31.3974,-102.3501,14.92,38,38,3.37,US,1706671144
4,4,siwa oasis,29.2041,25.5195,8.88,74,44,4.35,EG,1706671144


In [3]:
# Configure the map 
map = city_data_df.hvplot.points(x='Lng', y='Lat', geo = True, c='City', size='Humidity', hover_cols=['City'], tiles = 'OSM', alpha = 0.5)
map = map.opts(title='City Humidity Map', xlabel='Longitude', ylabel='Latitude')
# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
condition1 = city_data_df['Max Temp'] > 21
condition2 = city_data_df['Max Temp'] < 27
condition3 = city_data_df['Wind Speed'] < 4.5
condition4 = city_data_df['Cloudiness']  == 0

# Filter out those cities that satisfy the requirement listed in the instruction
ideal_cities = city_data_df.loc[condition1 & condition2 & condition3 & condition4]
ideal_cities
                       

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,barra de santo antonio,-9.4047,-35.5072,24.60,91,0,0.77,BR,1706671151
46,46,port elizabeth,-33.9180,25.5701,21.84,88,0,2.57,ZA,1706671154
131,131,coruripe,-10.1256,-36.1756,23.76,85,0,1.94,BR,1706671171
145,145,vila velha,-20.3297,-40.2925,26.97,94,0,3.60,BR,1706671174
189,189,sinazongwe,-17.2614,27.4618,23.05,78,0,0.68,ZM,1706671182
208,208,les avirons,-21.2418,55.3394,25.03,78,0,1.54,RE,1706671185
287,287,vallenar,-28.5708,-70.7581,23.50,61,0,0.68,CL,1706671199
337,337,kyaikkami,16.0800,97.5675,25.35,72,0,2.47,MM,1706671211
343,343,dondo,-19.6094,34.7431,26.73,87,0,1.60,MZ,1706671212
436,436,road town,18.4167,-64.6167,26.05,68,0,2.11,VG,1706671242


In [5]:
 # will drop all of the null values
hotel_df = ideal_cities.dropna()

# write print statement containing # of cities 
print(f'There are {len(hotel_df)} ideal cities for vacation')
hotel_df

There are 10 ideal cities for vacation


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,barra de santo antonio,-9.4047,-35.5072,24.60,91,0,0.77,BR,1706671151
46,46,port elizabeth,-33.9180,25.5701,21.84,88,0,2.57,ZA,1706671154
131,131,coruripe,-10.1256,-36.1756,23.76,85,0,1.94,BR,1706671171
145,145,vila velha,-20.3297,-40.2925,26.97,94,0,3.60,BR,1706671174
189,189,sinazongwe,-17.2614,27.4618,23.05,78,0,0.68,ZM,1706671182
208,208,les avirons,-21.2418,55.3394,25.03,78,0,1.54,RE,1706671185
287,287,vallenar,-28.5708,-70.7581,23.50,61,0,0.68,CL,1706671199
337,337,kyaikkami,16.0800,97.5675,25.35,72,0,2.47,MM,1706671211
343,343,dondo,-19.6094,34.7431,26.73,87,0,1.60,MZ,1706671212
436,436,road town,18.4167,-64.6167,26.05,68,0,2.11,VG,1706671242


In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "categories":"accomodation.hotel"
}

# Print a ge to follow up the hotel search
print("Beginning search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Beginning search
barra de santo antonio - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
coruripe - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
sinazongwe - nearest hotel: No hotel found
les avirons - nearest hotel: No hotel found
vallenar - nearest hotel: No hotel found
kyaikkami - nearest hotel: No hotel found
dondo - nearest hotel: No hotel found
road town - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
31,31,barra de santo antonio,-9.4047,-35.5072,24.60,91,0,0.77,BR,1706671151,No hotel found
46,46,port elizabeth,-33.9180,25.5701,21.84,88,0,2.57,ZA,1706671154,No hotel found
131,131,coruripe,-10.1256,-36.1756,23.76,85,0,1.94,BR,1706671171,No hotel found
145,145,vila velha,-20.3297,-40.2925,26.97,94,0,3.60,BR,1706671174,No hotel found
189,189,sinazongwe,-17.2614,27.4618,23.05,78,0,0.68,ZM,1706671182,No hotel found
208,208,les avirons,-21.2418,55.3394,25.03,78,0,1.54,RE,1706671185,No hotel found
287,287,vallenar,-28.5708,-70.7581,23.50,61,0,0.68,CL,1706671199,No hotel found
337,337,kyaikkami,16.0800,97.5675,25.35,72,0,2.47,MM,1706671211,No hotel found
343,343,dondo,-19.6094,34.7431,26.73,87,0,1.60,MZ,1706671212,No hotel found
436,436,road town,18.4167,-64.6167,26.05,68,0,2.11,VG,1706671242,No hotel found
